<a href="https://colab.research.google.com/github/AmeerHamza0220/Deep-Learning/blob/main/CNNBasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()
num_classes = 10
img_rows, img_cols = 32,32
num_channels = 3

In [ ]:
x_train.shape

In [ ]:
model=tf.keras.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(112,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01)))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(50,activation='relu',kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.01)))


model.add(tf.keras.layers.Dense(num_classes,activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=1e-2)

model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
               metrics=[tf.keras.metrics.sparse_categorical_accuracy])
callbacks = [tf.keras.callbacks.TensorBoard('./keras')]
model.fit(x_train, y_train, epochs=2, verbose=1, validation_data=(x_test, y_test), callbacks=callbacks)

Basic stuff using convulution and pooling

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/d/da/Epimachus_meyeri_-Papua_New_Guinea_-male-8.jpg

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
from skimage import io        # Package to simply read images

In [ ]:
from PIL import Image
image = Image.open('/content/Epimachus_meyeri_-Papua_New_Guinea_-male-8.jpg').convert('L').resize((600,600))


In [ ]:
image.shape

In [ ]:
plt.imshow(image, cmap=plt.cm.gray)

In [ ]:
image = tf.keras.preprocessing.image.img_to_array(image)

In [ ]:
image = tf.expand_dims(image, axis=0)
 #tensorflow work on batches 

In [ ]:

print("Tensor shape: {}".format(image.shape))

 we define a 3x3 filter (or kernel) commonly used to blur images (Gaussian blur):

In [ ]:
kernel = tf.constant([[1 / 16, 2 / 16, 1 / 16],
                      [2 / 16, 4 / 16, 2 / 16],
                      [1 / 16, 2 / 16, 1 / 16]], tf.float32, name="gaussian_kernel")

In [ ]:
kernel = tf.expand_dims(tf.expand_dims(kernel, axis=-1), axis=-1)
kernel.shape

In [ ]:
blurred_image = tf.nn.conv2d(image, kernel, strides=[1, 1, 1, 1], padding="SAME")

In [ ]:
blurred_res = blurred_image.numpy()
# We "unbatch" our result by selecting the first (and only) image; we also remove the depth dimension:
blurred_res = blurred_res[0, ..., 0]

plt.imshow(blurred_res, cmap=plt.cm.gray)

a kernel for contour detection is used. This kernel is defined as follows:

In [ ]:
kernel = tf.constant([[-1, -1, -1],
                      [-1,  8, -1],
                      [-1, -1, -1]], tf.float32, name="edge_kernel")
kernel = tf.expand_dims(tf.expand_dims(kernel, axis=-1), axis=-1)

In [ ]:
edge_image = tf.nn.conv2d(image, kernel, strides=[1, 2, 2, 1], padding="SAME")
edge_res = edge_image.numpy()[0, ..., 0]
plt.imshow(edge_res, cmap=plt.cm.gray)

For max-pooling and average-pooling, the values in each window are aggregated into a single output, applying respectively the max or averaging operation. Once again, we use the low-level TensorFlow API to reproduce the results shown in the chapter:

In [ ]:
avg_pooled_image = tf.nn.avg_pool(image, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
avg_res = avg_pooled_image.numpy()[0, ..., 0]
plt.imshow(avg_res, cmap=plt.cm.gray)

In [ ]:
max_pooled_image = tf.nn.max_pool(image, ksize=[1, 10, 10, 1], strides=[1, 2, 2, 1], padding="SAME")
max_res = max_pooled_image.numpy()[0, ..., 0]
plt.imshow(max_res, cmap=plt.cm.gray)

Simple convulutional network

In [ ]:
class LeNet(tf.keras.Model):
    def __init__(self,num_classes):
        super(LeNet, self).__init__()
        self.conv1=tf.keras.layers.Conv2D(20,kernel_size=5,padding="same",activation='relu')
        self.conv2=tf.keras.layers.Conv2D(30,kernel_size=5,activation='relu')
        self.maxpool=tf.keras.layers.MaxPooling2D(pool_size=2)
        self.flatten=tf.keras.layers.Flatten()
        self.dense1=tf.keras.layers.Dense(400,activation='relu')
        self.dense2=tf.keras.layers.Dense(num_classes,activation='relu')
    def call(self,input):
        x=self.conv1(input)
        x=self.maxpool(x)
        x=self.conv2(x)
        x=self.maxpool(x)
        x=self.flatten(x)
        x=self.dense1(x)
        x=self.dense2(x)
        return x

In [ ]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()
num_classes = 10
img_rows, img_cols = 32,32
num_channels = 3

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
class LeNet5(Model):
    
    def __init__(self, num_classes):
        """
        Initialize the model.
        :param num_classes:     Number of classes to predict from
        """
        super(LeNet5, self).__init__()
        # We instantiate the various layers composing LeNet-5:
        # self.conv1 = SimpleConvolutionLayer(6, kernel_size=(5, 5))
        # self.conv2 = SimpleConvolutionLayer(16, kernel_size=(5, 5))
        # ... or using the existing and (recommended) Conv2D class:
        self.conv1 = Conv2D(6, 5, padding='same', activation='relu')
        self.conv2 = Conv2D(16, 5, activation='relu')
        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu')
        self.dense2 = Dense(84, activation='relu')
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        """
        Call the layers and perform their operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x))) # dense layers
        return x

In [ ]:
input_shape=32,32,3


model = LeNet5(10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.sparse_categorical_accuracy])
callbacks = [
    # Callback to interrupt the training if the validation loss (`val_loss`) stops improving for over 3 epochs:
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    # Callback to log the graph, losses and metrics into TensorBoard (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True)]
# batched_input_shape = tf.TensorShape((None, *input_shape))
# model.build(input_shape=batched_input_shape)
# model.summary()

In [ ]:
x_train=tf.cast(x_train, tf.float32)
x_test=tf.cast(x_test, tf.float32)

history = model.fit(x_train, y_train,
                    batch_size=32, epochs=10, validation_data=(x_test, y_test), 
                    verbose=1,  # change to `verbose=1` to get a progress bar
                                # (we opt for `verbose=2` here to reduce the log size)
                    callbacks=callbacks)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

Improving above lenet with batchnorm and l1 regularization

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
class LeNet(Model):
    
    def __init__(self, num_classes):
        """
        Initialize the model.
        :param num_classes:     Number of classes to predict from
        """
        super(LeNet, self).__init__()
        # We instantiate the various layers composing LeNet-5:
        # self.conv1 = SimpleConvolutionLayer(6, kernel_size=(5, 5))
        # self.conv2 = SimpleConvolutionLayer(16, kernel_size=(5, 5))
        # ... or using the existing and (recommended) Conv2D class:
        self.conv1 = Conv2D(6, 5, padding='same', activation='relu',   kernel_regularizer=tf.keras.regularizers.l1(0.01),)
        self.bn1=tf.keras.layers.BatchNormalization()
        self.conv2 = Conv2D(16, 5, activation='relu',   kernel_regularizer=tf.keras.regularizers.l1(0.01),)
        self.bn2=tf.keras.layers.BatchNormalization()

        self.max_pool = MaxPooling2D(pool_size=(2, 2))
        self.flatten = Flatten()
        self.dense1 = Dense(120, activation='relu',   kernel_regularizer=tf.keras.regularizers.l1(0.01),)
        self.dense2 = Dense(84, activation='relu',   kernel_regularizer=tf.keras.regularizers.l1(0.01),)
        self.dense3 = Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        """
        Call the layers and perform their operations on the input tensors
        :param inputs:  Input tensor
        :return:        Output tensor
        """
        x = self.max_pool(self.conv1(inputs))        # 1st block
        x = self.max_pool(self.conv2(x))             # 2nd block
        x = self.flatten(x)
        x = self.dense3(self.dense2(self.dense1(x))) # dense layers
        return x

In [ ]:
input_shape=32,32,3


model = LeNet(10)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[tf.keras.metrics.sparse_categorical_accuracy])
callbacks = [
    # Callback to interrupt the training if the validation loss (`val_loss`) stops improving for over 3 epochs:
    tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
    # Callback to log the graph, losses and metrics into TensorBoard (saving log files in `./logs` directory):
    tf.keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True)]
# batched_input_shape = tf.TensorShape((None, *input_shape))
# model.build(input_shape=batched_input_shape)
# model.summary()

In [ ]:
x_train=tf.cast(x_train, tf.float32)
x_test=tf.cast(x_test, tf.float32)

history = model.fit(x_train, y_train,
                    batch_size=32, epochs=10, validation_data=(x_test, y_test), 
                    verbose=1,  # change to `verbose=1` to get a progress bar
                                # (we opt for `verbose=2` here to reduce the log size)
                    callbacks=callbacks)

Now we will implement Resnet

In [ ]:
class ConvWithBatchNorm(tf.keras.layers.Conv2D):
    # This is a custom layer that wraps a Conv2D layer with BatchNormalization and a ReLU activation
    def __init__(self,activation='relu', name='convbn',**kwargs):
        """
        :param activation: activation function to use
        :param name: name of the layer
        :param kwargs: arguments for the Conv2D layer
        """
        self.activation=tf.keras.layers.Activation(activation,name=name) if \
                          activation is not None else None
        super().__init__(activation=None, name=name + '_c', **kwargs)
        self.batch_norm = tf.keras.layers.BatchNormalization(axis=-1, name=name + '_bn')
    def call(self,input,training=None):
        x = super().call(inputs) #conv2d
        x=self.batch_norm(x,training=training)
        if self.activation is not None:
            x=self.activation(x)
        return x

import functools

class ResidualMerge(tf.keras.layers.Layer):
    """ Layer to merge the original tensor and the residual one in residual blocks"""

    def __init__(self, name='block', **kwargs):
        """
        Initialize the layer. 
        :param activation:   Activation function (name or callable)
        :param name:         Name suffix for the sub-layers.
        :param kwargs:       Optional parameters of tf.keras.layers.Conv2D
        """
        
        super().__init__(name=name)
        self.shortcut = None
        self.kwargs = kwargs
        
    def build(self, input_shape):
        x_shape = input_shape[0]
        x_residual_shape = input_shape[1]
        if x_shape[1] == x_residual_shape[1] and \
           x_shape[2] == x_residual_shape[2] and \
           x_shape[3] == x_residual_shape[3]:
            self.shortcut = functools.partial(tf.identity, name=self.name + '_shortcut')
        else:
            strides = (
                int(round(x_shape[1] / x_residual_shape[1])), # vertical stride
                int(round(x_shape[2] / x_residual_shape[2]))  # horizontal stride
            )
            x_residual_channels = x_residual_shape[3]
            self.shortcut = ConvWithBatchNorm(
                filters=x_residual_channels, kernel_size=(1, 1), strides=strides,
                activation=None, name=self.name + '_shortcut_c', **self.kwargs)
        

    def call(self, inputs):
        """
        Call the layer. 
        :param inputs:         Tuple of two input tensors to merge
        :return:               Merged tensor
        """
        x, x_residual = inputs
        
        x_shortcut = self.shortcut(x)
        x_merge = tf.add([x_shortcut, x_residual])
        return x_merge

class BasicResidualBlock(tf.keras.Model):
    # This is a basic residual block with two convolutional layers and a residual merge
    def __init__(self, filter=16,kernal_size=1,strides=1,activation='relu',kernel_initializer ='he_normal',kernel_regularizer=tf.keras.regularizers.l2(1e-4),name='block_basic',**kwargs):
        """
        :param filter:         Number of filters in the convolutional layers
        :param kernal_size:    Size of the convolutional kernels
        :param strides:        Stride of the convolutional layers
        :param activation:     Activation function (name or callable)
        :param kernel_initializer : Initializer for the convolutional kernels
        :param kernel_regularizer: Regularizer for the convolutional kernels
        :param name:           Name suffix for the sub-layers.
        :param kwargs:         Optional parameters of tf.keras.layers.Conv2D
        """
        super().__init__(name=name)
        self.conv1 = ConvWithBatchNorm(
            filters=filter, kernel_size=kernal_size, strides=strides,
            activation=activation, 
            kernel_initializer =kernel_initializer ,
            kernel_regularizer=kernel_regularizer,
            name=name + '_c1', **kwargs)
        self.conv2 = ConvWithBatchNorm(
            filters=filter, kernel_size=kernal_size, strides=strides,
            activation=activation,
            kernel_initializer =kernel_initializer ,
            kernel_regularizer=kernel_regularizer, name=name + '_c2', **kwargs)
        self.merge = ResidualMerge(name=name + '_merge',kernel_initializer =kernel_initializer ,kernel_regularizer=kernel_regularizer)
        self.activation = tf.keras.layers.Activation(activation,name=name + '_activation')
    def call(self,inputs,training=None):
        x = self.conv1(inputs,training=training)
        x = self.conv2(x,training=training)
        x = self.merge([inputs, x])
        x = self.activation(x)
        return x

class ResidualBlockWithBottleneck(tf.keras.Model):
    # This is a residual block with a bottleneck layer
    def __init__(self, filter=16,kernal_size=1,strides=1,activation='relu',kernel_initializer ='he_normal',kernel_regularizer=tf.keras.regularizers.l2(1e-4),name='block_bottleneck',**kwargs):
        """
        :param filter:         Number of filters in the convolutional layers
        :param kernal_size:    Size of the convolutional kernels
        :param strides:        Stride of the convolutional layers
        :param activation:     Activation function (name or callable)
        :param kernel_initializer : Initializer for the convolutional kernels
        :param kernel_regularizer: Regularizer for the convolutional kernels
        :param name:           Name suffix for the sub-layers.
        :param kwargs:         Optional parameters of tf.keras.layers.Conv2D
        """
        super().__init__(name=name)
        self.conv1 = ConvWithBatchNorm(
            filters=filter, kernel_size=kernal_size, strides=strides,
            activation=activation, 
            kernel_initializer =kernel_initializer ,
            kernel_regularizer=kernel_regularizer,
            name=name + '_c1', **kwargs)
        self.conv2 = ConvWithBatchNorm(
            filters=filter, kernel_size=kernal_size, strides=strides,
            activation=activation,
            kernel_initializer =kernel_initializer ,
            kernel_regularizer=kernel_regularizer, name=name + '_c2', **kwargs)
        self.conv3 = ConvWithBatchNorm(
            filters=filter*4, kernel_size=1, strides=1,
            activation=activation,
            kernel_initializer =kernel_initializer ,
            kernel_regularizer=kernel_regularizer, name=name + '_c3', **kwargs)
        self.merge = ResidualMerge(name=name + '_merge',kernel_initializer =kernel_initializer ,kernel_regularizer=kernel_regularizer)
        self.activation = tf.keras.layers.Activation(activation,name=name + '_activation')
        
    def call(self,inputs,training=None):
        x = self.conv1(inputs,training=training)
        x = self.conv2(x,training=training)
        x = self.conv3(x,training=training)
        x = self.merge([inputs, x])
        x = self.activation(x)
        return x


class MacroResidualBlock(tf.keras.models.Sequential):
    #multiple residual blocks as sequential model
    def __init__(self, block_class=ResidualBlockWithBottleneck,repititions=3,filter=16,kernal_size=1,strides=1,activation='relu',kernel_initializer ='he_normal',kernel_regularizer=tf.keras.regularizers.l2(1e-4),name='block_macro',**kwargs):
        
        super.__init__([block_class(filter=filter,kernal_size=kernal_size,strides=strides,activation=activation,kernel_initializer =kernel_initializer ,kernel_regularizer=kernel_regularizer,name=name+'_block_'+str(i),**kwargs) for i in range(repititions)],name=name)

class ResNet(tf.keras.Sequential):
    # ResNet model
    def __init__(self, input_shape,num_classes=1000,block_class=ResidualBlockWithBottleneck,repititions=3,activation='relu',kernel_initializer ='he_normal',kernel_regularizer=tf.keras.regularizers.l2(1e-4),name='resnet',**kwargs):
        """
        :param input_shape:     Shape of the input image
        :param num_classes:     Number of classes in the dataset
        :param block_class:     Class of the residual block
        :param repititions:     Number of residual blocks
        :param activation:      Activation function (name or callable)
        :param kernel_initializer : Initializer for the convolutional kernels
        :param kernel_regularizer: Regularizer for the convolutional kernels
        :param name:            Name suffix for the sub-layers.
        :param kwargs:          Optional parameters of tf.keras.layers.Conv2D
        """
        filter=64
        strides=2
        super.__init__(
           #initial conv and pool layer
               [tf.keras.layers.Input(shape=input_shape),
              ConvWithBatchNorm(
                    filters=filter, kernel_size=7, strides=2,
                    activation='relu',
                    kernel_initializer =kernel_initializer ,
                    kernel_regularizer=kernel_regularizer,
                    padding='same',
                    name='conv'),
                tf.keras.layers.MaxPool2D(pool_size=3, strides=strides, 
                padding='same')]+
            #residual blocks
            [block_class(filter=min(filters * (2 ** i), 1024),kernal_size=kernal_size,strides=strides if i !=0 else 1,activation=activation,kernel_initializer =kernel_initializer ,kernel_regularizer=kernel_regularizer,name=name+'_block_'+str(i),**kwargs) for i in range(repititions)]+
            #final classification layer
            [tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(num_classes,
            kernel_initializer =kernel_initializer ,
            activation='softmax',
            name='classification')])


In [ ]:
class ResNet18(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet18',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=BasicResidualBlock, repetitions=(2, 2, 2, 2),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)
        
class ResNet34(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet34',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=BasicResidualBlock, repetitions=(3, 4, 6, 3),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)
        
class ResNet50(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet50',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=ResidualBlockWithBottleneck, repetitions=(3, 4, 6, 3),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)

class ResNet101(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet101',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=ResidualBlockWithBottleneck, repetitions=(3, 4, 23, 3),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)

class ResNet152(ResNet):
    def __init__(self, input_shape, num_classes=1000, name='resnet152',
                 kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(1e-4)):
        super().__init__(input_shape, num_classes, 
                         block_class=ResidualBlockWithBottleneck, repetitions=(3, 8, 36, 3),
                         kernel_initializer=kernel_initializer, kernel_regularizer=kernel_regularizer)

In [ ]:
input_shape = [224, 224, 3] 
num_classes=1000
model = ResNet50(input_shape, num_classes)
model.summary(line_length=80, positions=[.5, .85, 1.])